In [24]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup

In [25]:
url_list = [{"https://www.burgerbaeclothing.com/collections/basics": 3},
       {"https://www.burgerbaeclothing.com/collections/co-ord-set-1": 2},
       {"https://www.burgerbaeclothing.com/collections/t-shirt": 5},
       {"https://www.burgerbaeclothing.com/collections/baby-tees-starting-from-399": 4},
       {"https://www.burgerbaeclothing.com/collections/tanks": 3},
       {"https://www.burgerbaeclothing.com/collections/tops": 1},
       {"https://www.burgerbaeclothing.com/collections/bottoms": 2},
       {"https://www.burgerbaeclothing.com/collections/dresses":1},
       {"https://www.burgerbaeclothing.com/collections/accessories-1":1},
       {"https://www.burgerbaeclothing.com/collections/hoodies":3},
       {"https://www.burgerbaeclothing.com/collections/sweatshirt": 2},
       {"https://www.burgerbaeclothing.com/collections/winterwear": 2}
       ]

In [26]:
def extract_prices_from_list(price_info) :
    if "Regular price" in price_info: 
        start = price_info.find("Sale price₹") + len("Sale price₹")
        end = price_info.find("Regular price")
        sale_price = price_info[start:end].strip()

        start = price_info.find("Regular price₹") + len("Regular price₹")
        regular_price = price_info[start:].strip()
    else :
        sale_price = ''
        start = price_info.find("Sale price₹") + len("Sale price₹")
        regular_price = price_info[start:].strip()
    return [sale_price, regular_price]

In [27]:
product_names = []
sale_prices = []
original_prices = []
product_primary_images = []
product_seconday_images = []
sizes = []
categories = []
product_descriptions = []
product_urls = []

for url_dict in url_list:
    for url, pages in url_dict.items():
        print(url, pages)
        for i in range(pages):
            print(url, i)
            prefix = '?page=' + str(i+1)
            if (i == 0) : prefix = ""
            url_prefix = url + prefix
            req = r.get(url_prefix)
            soup = BeautifulSoup(req.text, 'html.parser')
            product_containers = soup.find_all('div', class_='product-grid-item')
            # print(product_containers[0])
            j = 0
            for product in product_containers: 
                j = j+1
                child_url = product.find('a', class_='product-grid-item--link').get('href').split('/')[4]
                child_url = url + '/products/' + child_url
                req_child = r.get(child_url)
                soup_child = BeautifulSoup(req_child.text, 'html.parser')
                child_images = []
                child_images_div = soup_child.find_all('div', class_='media__thumb')
                for child_div in child_images_div :
                    child_images.append(child_div.find('img').get('src').lstrip('//'))
                sizes_span = soup_child.find_all('span',class_='radio__button')
                sizes_prod = []
                for size in sizes_span:
                    sizes_prod.append(size.text.strip())
                sizes.append(sizes_prod)
                product_info = product.find('div', class_='grid__caption text-center').text.strip()
                product_names.append(product_info.split('\n')[0])
                product_urls.append(child_url)
                product_primary_images.append(product.find('img', class_='img-object-cover').get('src').lstrip('//'))
                # print(product_info.split('\n')[0])
                product_seconday_images.append(child_images)
                if product.find('p', class_='price_wrapper on-sale') :
                    price_info = product.find('p', class_='price_wrapper on-sale').text.strip().split('.')
                    sale_prices.append(price_info[1].strip())
                    original_prices.append(price_info[3].strip())
                else : 
                    sale_prices.append('')
                    original_prices.append(price_info[3].strip())
                categories.append(soup.find('h1', class_='h3 standard__heading collection__heading__text heading-size-10').text.strip())
                product_descriptions.append([])
                

# df = pd.DataFrame({
#     'Product Name': product_names,
#     'Price': prices,
#     'Variant': item_variants,
#     'Category': item_list_names,
#     'ImageURL': image_urls
# })


df = pd.DataFrame({
    'Category': categories,
    'Product Name': product_names,
    'Product Link': product_urls,
    'Sale Price': sale_prices,
    'Original Price': original_prices,
    'Sizes': sizes,
    'Primary Image': product_primary_images,
    'Product Secondary Images': product_seconday_images,
    'Product URLs': product_urls,
    'Product Description': product_descriptions
})

df.to_csv('burger_bae.csv', index=False)
print("Data scraped and saved successfully!")
        

https://www.burgerbaeclothing.com/collections/basics 3
https://www.burgerbaeclothing.com/collections/basics 0


KeyboardInterrupt: 